In [91]:
import matplotlib.pyplot as plt
from google.colab import drive
import pandas as pd
from datetime import datetime
import numpy as np

In [ ]:
drive.mount('/content/gdrive')

In [94]:
artData = pd.read_csv("/content/gdrive/MyDrive/artData.csv")

In [95]:
artData.head()

,UID,category,editModifyDate,startDate,endDate,hitRate,time,locationName,onSale,latitude,longitude,endTime
0,5aa90765c6355e63fc0cf750,16,NaN,2112/01/29,2112/01/29,0,2112/01/29 00:00:00,NaN,N,NaN,NaN,NaN
1,5b3dd544aaa378d7ca9a2e9a,6,2018/07/06 14:25:35,2021/12/07,2028/12/31,1757,2021/12/07 09:00:00,人類文化廳二樓,Y,24.157234,120.666060,2028/12/31 17:00:00
2,5b3dd544aaa378d7ca9a2e9a,4,2018/07/06 14:25:35,2021/12/07,2028/12/31,1757,2021/12/07 09:00:00,人類文化廳二樓,Y,24.157234,120.666060,2021/12/07 17:00:00
3,5c7e1bcdaaa375d860933d3a,6,2019/03/06 17:19:54,2019/03/05,2024/03/05,1792,2019/03/05 09:00:00,國立自然科學博物館,Y,24.157255,120.666184,2024/03/05 17:00:00
4,5c7e1bcdaaa375d860933d3a,4,2019/03/06 17:19:54,2019/03/05,2024/03/05,1792,2019/03/05 09:00:00,國立自然科學博物館,Y,24.157255,120.666184,2024/03/05 17:00:00


### **Data Cleaning**

In [74]:
artData = artData[artData["startDate"] != "2112/01/29"].reset_index(drop = True)

### **Feature Transformation**

In [75]:
yearArry = []
for i in range(len(artData)):
  if artData["startDate"][i].split("/")[0] in ["2019", "2020", "2021"]:
    yearArry.append("Past")
  else:
    yearArry.append("Future")
artData["YearType"] = yearArry

In [ ]:
for i in range(len(artData)):
  artData["startDate"][i] = datetime.strptime(artData["startDate"][i], "%Y/%m/%d")
for i in range(len(artData)):
  artData["endDate"][i] = datetime.strptime(artData["endDate"][i], "%Y/%m/%d")

In [77]:
period = []
for i in range(len(artData)):
  period.append((artData["endDate"][i] - artData["startDate"][i]).days)
artData["Period"] = period

In [78]:
# 篩選出有重複類別資料
dfUIDcount = artData[["UID"]].groupby("UID", as_index = False).size()
dfMultipleUID = dfUIDcount[dfUIDcount["size"] > 1]
dfFilter = artData[artData["UID"].isin(dfMultipleUID["UID"])].reset_index()

In [ ]:
# 重複類別欄位整理
combCat = []
for i in range(len(dfFilter)-1):
  if dfFilter["UID"][i] == dfFilter["UID"][i+1]:
    combCat.append(str(dfFilter["category"][i]) + "," + str(dfFilter["category"][i+1]))
  elif dfFilter["UID"][i] != dfFilter["UID"][i+1]:
    combCat.append("")
combCat.append("")
dfFilter["combCat"] = combCat
for i in range(len(dfFilter)):
  if dfFilter["combCat"][i] == "":
    dfFilter["combCat"][i] = dfFilter["combCat"][i-1]

In [80]:
dfFilter[["combCat"]].groupby("combCat", as_index = False).size() # 觀察各類別組合數量

,combCat,size
0,"1,200",2
1,"1,4",4
2,"14,200",1
3,"15,200",4
4,"15,4",8
5,"19,200",10
6,"19,4",6
7,"2,200",1
8,"2,4",4
9,"200,4",14


In [81]:
dfFilter[["combCat", "hitRate"]].groupby("combCat", as_index = False).mean("hitRate") # 各類別組合平均點擊率

,combCat,hitRate
0,"1,200",1.000000
1,"1,4",203.500000
2,"14,200",12.000000
3,"15,200",3.500000
4,"15,4",5.500000
5,"19,200",23.200000
6,"19,4",1318.333333
7,"2,200",9.000000
8,"2,4",87.500000
9,"200,4",8.000000
